In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import paddle
print(paddle.__version__)
import paddlets
print(paddlets.__version__)

from paddlets.utils import backtest
from paddlets import TSDataset, TimeSeries
from paddlets.xai.post_hoc.shap_explainer import ShapExplainer
from paddlets.datasets.repository import get_dataset

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#加载数据 再训练
data = pd.read_csv('/home/Zone9-power.csv')

# 将时间戳列转换为日期时间格式
data['TIMESTAMP'] = pd.to_datetime(data['TIMESTAMP'])

# 将数据按照时间顺序排序
data.sort_values('TIMESTAMP', inplace=True)

# 划分数据集
train_dataset = data[data['TIMESTAMP'].dt.year == 2012]
val_dataset = data[(data['TIMESTAMP'].dt.year == 2013) & (data['TIMESTAMP'].dt.month == 1)]
test_dataset = train_dataset

# 在这里定义模型的out_chunk_len
model_out_chunk_len = 4

train_dataset_target = train_dataset[:-model_out_chunk_len]
val_dataset_target = val_dataset[:-model_out_chunk_len]
test_dataset_target = test_dataset[:-model_out_chunk_len]


# 检查数据集的大小
print("训练集大小:", len(train_dataset))
print("验证集大小:", len(val_dataset))
print("测试集大小:", len(test_dataset))

# 打印前五行数据
print(train_dataset.head())
print(val_dataset.head())
print(test_dataset.head())

In [ ]:
#加载数据
data = pd.read_csv('/home/Zone1-power.csv')

# 将时间戳列转换为日期时间格式
data['TIMESTAMP'] = pd.to_datetime(data['TIMESTAMP'])

# 将数据按照时间顺序排序
data.sort_values('TIMESTAMP', inplace=True)

# 划分数据集
train_dataset = data[data['TIMESTAMP'].dt.year == 2012]
val_dataset = data[(data['TIMESTAMP'].dt.year == 2013) & (data['TIMESTAMP'].dt.month == 1)]
test_dataset = data[(data['TIMESTAMP'].dt.year == 2013) & (data['TIMESTAMP'].dt.month == 2)]

# 检查数据集的大小
print("训练集大小:", len(train_dataset))
print("验证集大小:", len(val_dataset))
print("测试集大小:", len(test_dataset))

# 打印前五行数据
print(train_dataset.head())
print(val_dataset.head())
print(test_dataset.head())

In [ ]:
#数据转换
from paddlets import TSDataset
import matplotlib.pyplot as plt

target_cov_dataset1 = TSDataset.load_from_dataframe(
    train_dataset,
    time_col='TIMESTAMP',
    target_cols='TARGETVAR',
    freq='1h'
    )

target_cov_dataset2 = TSDataset.load_from_dataframe(
    val_dataset,
    time_col='TIMESTAMP',
    target_cols='TARGETVAR',
    freq='1h'
    )

target_cov_dataset3 = TSDataset.load_from_dataframe(
    test_dataset,
    time_col='TIMESTAMP',
    target_cols='TARGETVAR',
    freq='1h'
    )

train_dataset = target_cov_dataset1
val_dataset = target_cov_dataset2
test_dataset = target_cov_dataset3#分割数据集

train_dataset.plot(add_data=[val_dataset,test_dataset], labels=['Val', 'Test'])
plt.savefig('fig1.svg',format='svg')

In [ ]:
from informer import *
InF = InformerModel(
    in_chunk_len = 24*3,
    out_chunk_len = 4,
    max_epochs = 50,
    d_model = 64,
    nhead = 8,
    start_token_len = 6,
    ffn_channels = 1024,
    num_encoder_layers = 2,
    num_decoder_layers = 1,
    activation = "relu",
    verbose = 1,
    patience = 4
    )
InF.fit(train_dataset, val_dataset)
subset_test_pred_dataset = InF.predict(test_dataset)#模型预测
subset_test_dataset, _ = test_dataset.split(len(subset_test_pred_dataset.target))#截取某天进行预测
subset_test_dataset.plot(add_data=subset_test_pred_dataset, labels=['Pred'])
from paddlets.utils import backtest
score, preds_data= backtest(
    data=test_dataset,
    model=InF,
    return_predicts = True)
print(f"mae: {score}")
test_dataset.plot(add_data=preds_data,labels="backtest")

In [48]:
pred = preds_data.to_dataframe()
pred.to_csv("ts1111.csv")
true = test_dataset.to_dataframe()
true.to_csv("ts2222.csv")

pred=pd.read_csv('ts1111.csv')
pred=pred.rename(columns={pred.columns[0]: 'TIMESTAMP',pred.columns[1]: 'pred'})
true=pd.read_csv('ts2222.csv')
df_merged = pd.merge(pred, true, on="TIMESTAMP", how="inner")
df_merged.to_csv("trans9.csv")